In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
project_id = "perceptive-ivy-290216"
df_bq = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [14]:
GP=df_bq[((df_bq["GP"]=="Monaco Grand Prix") & (df_bq["Year"]==2024))]
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1998/2431489362.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
12262,0 days 01:39:03.085000,ALB,23,0 days 00:41:34.817000,1,2.0,0 days 01:36:41.722000,NaT,0 days 00:00:26.143000,0 days 00:00:40.862000,0 days 00:01:15.250000,0 days 01:37:07.085000,0 days 01:37:47.787000,0 days 01:39:03.226000,173.0,181.0,118.0,260.0,False,HARD,1.0,True,Williams,0 days 00:55:05.680000,NaT,254,9.0,False,,False,False,2024,Monaco Grand Prix
160527,0 days 01:40:30.919000,ALB,23,0 days 00:01:27.834000,2,2.0,NaT,NaT,0 days 00:00:25.399000,0 days 00:00:39.935000,0 days 00:00:22.500000,0 days 01:39:28.435000,0 days 01:40:08.370000,0 days 01:40:30.870000,158.0,176.0,255.0,277.0,True,HARD,2.0,True,Williams,0 days 01:39:03.085000,NaT,1,9.0,False,,False,True,2024,Monaco Grand Prix
160416,0 days 01:41:54.205000,ALB,23,0 days 00:01:23.286000,3,2.0,NaT,NaT,0 days 00:00:21.826000,0 days 00:00:39.210000,0 days 00:00:22.250000,0 days 01:40:52.696000,0 days 01:41:31.906000,0 days 01:41:54.156000,181.0,166.0,263.0,278.0,True,HARD,3.0,True,Williams,0 days 01:40:30.919000,NaT,1,9.0,False,,False,True,2024,Monaco Grand Prix
160456,0 days 01:43:17.574000,ALB,23,0 days 00:01:23.369000,4,2.0,NaT,NaT,0 days 00:00:22.133000,0 days 00:00:39.122000,0 days 00:00:22.114000,0 days 01:42:16.289000,0 days 01:42:55.411000,0 days 01:43:17.525000,167.0,167.0,265.0,281.0,False,HARD,4.0,True,Williams,0 days 01:41:54.205000,NaT,1,9.0,False,,False,True,2024,Monaco Grand Prix
15329,0 days 01:44:40.990000,ALB,23,0 days 00:01:23.416000,5,2.0,NaT,NaT,0 days 00:00:22.168000,0 days 00:00:39.171000,0 days 00:00:22.077000,0 days 01:43:39.693000,0 days 01:44:18.864000,0 days 01:44:40.941000,172.0,175.0,261.0,274.0,False,HARD,5.0,True,Williams,0 days 01:43:17.574000,NaT,1,9.0,False,,False,True,2024,Monaco Grand Prix


In [15]:
# GP=GP[GP["Driver"]!="GAS"]

In [16]:
fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
#             markers=True,
        title="Race Position Ladder for the {} {}".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=1000, width=1200, template="plotly_dark",
             color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          },
        )

fig.update_layout(
    title_x=0.5,
    yaxis_range=[1, 21],
    margin=dict(l=10, r=5, t=55, b=20),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1
    ),

    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),

    margin_pad=10
)
fig.update_yaxes(autorange="reversed")

# fig.update_layout(
    # updatemenus=[
    #     {
    #         "buttons": [
    #             {
    #                 "label": t.name,
    #                 "method": "restyle",
    #                 "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
    #             }
    #             for t in fig.data
    #         ]
    #     }
    # ]
# )

In [13]:
fig.write_html("{} {} Race Ladder.html".format(year,gp))

In [20]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
GP=GP[GP["LapTime (s)"]<500]

In [21]:
fig_all_laps=px.scatter(GP, 
        x="LapNumber", 
        y="LapTime (s)", 
        color='Driver',
        title="All Lap Times for the {} {}".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=700,
          width=1200, 
        template="plotly_dark",
        color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          },
        )
fig_all_laps.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow",
    # margin=dict(l=5, r=5, t=35, b=5),
    ),
    updatemenus=[
        {
            "buttons": [
                {
                    "label": t.name,
                    "method": "restyle",
                    "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
                }
                for t in fig.data
            ]
        }
    ],
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 2
    ),

    margin_pad=10
)

In [22]:
fig_all_laps.write_html("{} {} All Lap Times.html".format(year,gp))

In [119]:
# GP=GP.sort_values(['LapNumber', "Driver"])
# GP.tail()

In [120]:
# fig2=px.scatter(GP,
#         x="LapNumber",
#         y="Position",
#         color='DriverNumber',
#         animation_frame="LapNumber",
#         title="Race Position Ladder for the {} {}".format(year,gp),
#         # animation_group='LapNumber',
#         range_x=[0,len(GP['LapNumber'].unique())+1],
#         text='Driver',
#         range_y=[0,20],
#         hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
#         height=1000, width=1800, template="plotly_dark",
#            color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           },
#          )


# fig2.update_yaxes(autorange="reversed")

# fig.update_traces(textposition='middle right')

# fig2.update_layout(
#     title_x=0.5,
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="White"
#     ),

#     yaxis = dict(
#     tickmode = 'linear',
#     tick0 = 0,
#     dtick = 1
#     ),

#     margin_pad=10
# )

# fig2.update(layout_coloraxis_showscale=False)
# # for x,y in zip(GP[GP["Driver"]=="SAI"].Driver, GP[GP["Driver"]=="SAI"].Position):
# #   for png in (Path.cwd().joinpath("f1").glob("*.png")):
# #     if str.split(str.split(str(png),".")[0],"/")[3]==x:
# #       image=str(png)
# #       fig2.add_layout_image(
# #           x=x,
# #           y=y+4,
# #           source=Image.open(image),
# #           xref="x",
# #           yref="y",
# #           sizex=5,
# #           sizey=5,
# #           xanchor="center",
# #           yanchor="middle",
# #       )
# fig2.show()

In [111]:
# fig2.write_html("{} {} Race Animation.html".format(year,gp))

# Fastest Driver in each lap

In [86]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["RK"] = GP_Rk.groupby("LapNumber")["LapTime"].rank(method="dense", ascending=True)
GP_FASTEST_LAPS=GP_Rk[GP_Rk["RK"]==1.0].sort_values("LapNumber")
GP_FASTEST_LAPS=GP_FASTEST_LAPS[["Time","Driver","Team","LapTime","LapNumber","Stint","PitOutTime","PitInTime","Sector1Time","Sector2Time","Sector3Time","SpeedI1","SpeedI2","TyreLife","TrackStatus","Position","Year","GP","RK"]]
GP_FASTEST_LAPS.head()

,Time,Driver,Team,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,TyreLife,TrackStatus,Position,Year,GP,RK
543798,0 days 00:58:40.015000,RUS,Mercedes,00:01:43.723000,1,1.0,NaT,NaT,NaT,0 days 00:00:31.645000,0 days 00:00:37.671000,190.0,264.0,1.0,12,1.0,2024,Canadian Grand Prix,1.0
108881,0 days 01:00:16.571000,RUS,Mercedes,00:01:36.556000,2,1.0,NaT,NaT,0 days 00:00:28.287000,0 days 00:00:30.597000,0 days 00:00:37.672000,198.0,264.0,2.0,1,1.0,2024,Canadian Grand Prix,1.0
74979,0 days 01:02:01.046000,MAG,Haas F1 Team,00:01:36.134000,3,1.0,NaT,NaT,0 days 00:00:27.936000,0 days 00:00:31.214000,0 days 00:00:36.984000,198.0,268.0,3.0,1,4.0,2024,Canadian Grand Prix,1.0
364215,0 days 01:03:35.039000,MAG,Haas F1 Team,00:01:33.993000,4,1.0,NaT,NaT,0 days 00:00:27.334000,0 days 00:00:29.992000,0 days 00:00:36.667000,215.0,265.0,4.0,1,4.0,2024,Canadian Grand Prix,1.0
129219,0 days 01:05:04.364000,VER,Red Bull Racing,00:01:33.859000,5,1.0,NaT,NaT,0 days 00:00:27.574000,0 days 00:00:30.293000,0 days 00:00:35.992000,200.0,267.0,5.0,1,2.0,2024,Canadian Grand Prix,1.0
